In [1]:
import pandas as pd
import numpy as np

In [2]:
!head -n 2 google*
!wc -l google*

==> googleplaystore.csv <==
App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up

==> googleplaystore_user_reviews.csv <==
App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
10 Best Foods for You,"I like eat delicious food. That's I'm cooking food myself, case ""10 Best Foods"" helps lot, also ""Best Before (Shelf Life)""",Positive,1.0,0.5333333333333333
  10842 googleplaystore.csv
  64296 googleplaystore_user_reviews.csv
  75138 total


In [3]:
gps = pd.read_csv("googleplaystore.csv", index_col = 0)
gps_ur = pd.read_csv("googleplaystore_user_reviews.csv", index_col = 0)

In [4]:
pd.unique(gps["Size"].str.replace(r'[0-9]+', ''))

array(['M', '.M', 'Varies with device', 'k', '.k'], dtype=object)

In [5]:
## SOLVED
# gps[gps['Size'].str.contains("\+")] # Una sola riga
# A questa riga manca originariamente la seconda colonna (Category)
# di conseguenza le colonne risultano shiftate
# Modifichiamo manualmente il csv aggiungendo una virgola in piu'

In [6]:
gps[gps['Size'].str.contains("Varies with device")].shape

(1695, 12)

In [7]:
gps.loc[gps['Size'].str.contains("\+|Varies with device"), "Size"] = "-1"

In [8]:
sz = pd.DataFrame(gps.Size.str.split('.').tolist())
print(sz.shape) # no more than two columns --> no more than two dots
sz[1].unique() # always just one number after the dot -> no thousands

(10841, 2)


array([None, '7M', '8M', '6M', '1M', '5M', '2M', '0M', '4M', '9M', '3M',
       '5k'], dtype=object)

In [9]:
# Only one app has decimal size with kilobytes
gps[gps['Size'].str.contains("\.[0-9]k")] # the only one

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Essential Resources,LIBRARIES_AND_DEMO,4.6,237,8.5k,"50,000+",Free,0,Everyone,Libraries & Demo,"September 27, 2017",1.02,7.1 and up


In [10]:
gps["size_n"] = pd.to_numeric(gps.Size.str.extract('(^\d+(\.\d)?)')[0])
# Non perdiamo dati, quelli con size non determinabile
# sono sempre quelli (1695)
sum(pd.isna(gps["size_n"]))

1695

In [11]:
def flag_size(series):
    if (series.str.contains("k")):
        return 1024
    elif (series.str.contains("M")):
        return 1024 * 10 ** 3
    else:
        return 0
#gps['size_mag'] = gps.apply(flag_df_size, axis = 1)
#gps['size_mag'] = gps['Size'].apply(flag_size, axis = 1)
gps['size_mag'] = pd.np.where(gps.Size.str.contains("k"), 1024,
                              pd.np.where(gps.Size.str.contains("M"), 1024 ** 2,
                                          0))


In [12]:
gps["Size"] = gps.size_n * gps.size_mag
df1 = gps.drop(gps.columns[[-1, -2]], axis = 1, inplace = False)
df1

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


In [13]:
pd.unique(df1.Installs)
#df1[df1.Installs.str.contains("Free")] # Una sola riga | solved

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

In [14]:
df1.Installs = df1.Installs.str.replace(r'[\,\+]', '').astype(int)
df1

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


In [15]:
df1.replace({'Varies with device' : None}, inplace = True)
df1

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",None,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


In [16]:
df1["Ver"] = df1["Current Ver"].str.extract('(\d+(\.\d+)*)')[0]
df1["Ver_compatib"] = df1["Android Ver"].str.extract('(\d+(\.\d+)*)')[0]

In [17]:
df1

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Ver,Ver_compatib
App,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,1.0.0,4.0.3
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,2.0.0,4.0.3
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,1.2.4,4.0.3
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",None,4.2 and up,NaN,4.2
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,1.1,4.4
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,1.0,2.3
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,1.1,4.0.3
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,6.1.61.1,4.2
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,2.9.2,3.0


In [18]:
df1["Current Ver"] = df1.Ver
df1["Android Ver"] = df1.Ver_compatib
df2 = df1.drop(df1.columns[[-1, -2]], axis = 1, inplace = False)

In [19]:
df2

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0


In [20]:
df2[df2.index.duplicated(keep = False)].sort_index().shape # total duplicates
df2[df2.index.duplicated(keep = "last")].sort_index().shape # unique duplicates

(1181, 12)

In [21]:
# Remove duplicates, keep last (most recent ?)
df3 = df2[~df2.index.duplicated(keep = 'last')] # 10841 - 1181 = 9660
df3

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3250585.6,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3


In [22]:
print(df3.shape)
cat = df3.groupby('Category').size()
sum(cat) # One category is missing 
df3.loc[df3['Category'].isnull()] # sempre quella di prima !

(9660, 12)


,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3145728.0,1000,Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0


(6)-(7) Number of apps and average ratings for each category

In [23]:
agg1 = df3.groupby("Category").aggregate({"Rating": 'mean', "Category": 'count'}).rename(columns = {"Rating": "Rating_avg", "Category": "count"})
print(agg1["count"].sum())
agg1

9659


,Rating_avg,count
Category,,
ART_AND_DESIGN,4.359322,61
AUTO_AND_VEHICLES,4.190411,85
BEAUTY,4.278571,53
BOOKS_AND_REFERENCE,4.344970,222
BUSINESS,4.096565,419
COMICS,4.181481,56
COMMUNICATION,4.121401,316
DATING,3.980451,170
EDUCATION,4.351485,102


In [24]:
df3[df3.index.str.contains("Pixel Draw")]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4


In [25]:
genres_df = pd.DataFrame({"Genre": df3.Genres.str.split(";", expand = True).stack().unique()})
# genres_df = pd.DataFrame({"Genre": pd.unique(df3.Genres)})
genres_df

,Genre
0,Art & Design
1,Creativity
2,Auto & Vehicles
3,Beauty
4,Books & Reference
5,Business
6,Comics
7,Communication
8,Dating
9,Education


In [26]:
bridging_df = df3.Genres.str.split(";").apply(pd.Series).reset_index() \
    .melt(id_vars = "App", var_name = "foo", value_name = "Genre") \
    .drop("foo", axis = 1) \
    .dropna()
bridging_df

,App,Genre
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design
1,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design
2,Sketch - Draw & Paint,Art & Design
3,Pixel Draw - Number Art Coloring Book,Art & Design
4,Paper flowers instructions,Art & Design
5,Smoke Effect Photo Maker - Smoke Editor,Art & Design
6,Infinite Painter,Art & Design
7,Garden Coloring Book,Art & Design
8,Kids Paint Free - Drawing Fun,Art & Design
9,Text on Photo - Fonteee,Art & Design


In [27]:
bridging_df.groupby("App").count().sort_values(by = ["Genre", "App"], ascending = [False, True])

,Genre
App,
ABC Kids - Tracing & Phonics,2
ABC Preschool Free,2
ABCmouse.com,2
AJ Academy: Amazing Animals,2
Adult Glitter Color by Number Book - Sandbox Pages,2
Ag Across America,2
Alif Ba Ta Song - Arabic Kids,2
"Alizay, pirate girl",2
All-in-One Mahjong 3,2


In [28]:
genres_dummy = df3.iloc[:, df3.columns.get_loc('Genres')].str.get_dummies(sep=';').astype("bool")
genres_dummy.add_prefix("genre_")
genres_dummy

,Action,Action & Adventure,Adventure,Arcade,Art & Design,Auto & Vehicles,Beauty,Board,Books & Reference,Brain Games,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
App,,,,,,,,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Sketch - Draw & Paint,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Pixel Draw - Number Art Coloring Book,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Paper flowers instructions,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Smoke Effect Photo Maker - Smoke Editor,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Infinite Painter,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Garden Coloring Book,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Kids Paint Free - Drawing Fun,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
print("No duplicates df shape: {}\nDummy genres df shape: {}".format(df3.shape, genres_dummy.shape))
genres_dummy.sum(axis = 1).sort_values(ascending = False) # missing genre in "Life Made WI-Fi*"

No duplicates df shape: (9660, 12)
Dummy genres df shape: (9660, 53)


App
Unicorn Pokez - Color By Number                       2
Hactar Go                                             2
Steam: Rails to Riches                                2
No.Diamond – Colors by Number                         2
SweetLand — Family Board Game                         2
EP Chain Reaction                                     2
World Racers family board game                        2
All-in-One Mahjong 3                                  2
Equestria Girls                                       2
Math games for kids : times tables training           2
LEGO® Juniors Create & Cruise                         2
Ag Across America                                     2
LEGO ® Batman: Beyond Gotham                          2
All-in-One Mahjong 3 FREE                             2
Children Educational Game Full                        2
The Great Wobo Escape Ep.1                            2
Discovery K!ds Play!                                  2
Math games for kids : times tables - AB Math

In [30]:
df4 = df3.join(genres_dummy)
df4

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
App,,,,,,,,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",...,False,False,False,False,False,False,False,False,False,False
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",...,False,False,False,False,False,False,False,False,False,False
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",...,False,False,False,False,False,False,False,False,False,False
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",...,False,False,False,False,False,False,False,False,False,False
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",...,False,False,False,False,False,False,False,False,False,False
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",...,False,False,False,False,False,False,False,False,False,False
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",...,False,False,False,False,False,False,False,False,False,False
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",...,False,False,False,False,False,False,False,False,False,False
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3250585.6,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",...,False,False,False,False,False,False,False,False,False,False


In [31]:
df4

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
App,,,,,,,,,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",...,False,False,False,False,False,False,False,False,False,False
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",...,False,False,False,False,False,False,False,False,False,False
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",...,False,False,False,False,False,False,False,False,False,False
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",...,False,False,False,False,False,False,False,False,False,False
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",...,False,False,False,False,False,False,False,False,False,False
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",...,False,False,False,False,False,False,False,False,False,False
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",...,False,False,False,False,False,False,False,False,False,False
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",...,False,False,False,False,False,False,False,False,False,False
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3250585.6,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",...,False,False,False,False,False,False,False,False,False,False


In [32]:
df5 = bridging_df.merge(df3['Rating'].reset_index(), on = "App")
df5.dropna().groupby("Genre").mean().sort_values(by = "Rating", ascending = False) # Events with avg rating of 4.436

,Rating
Genre,
Events,4.435556
Puzzle,4.370732
Brain Games,4.358730
Art & Design,4.350000
Books & Reference,4.343275
Personalization,4.332215
Word,4.318182
Creativity,4.312903
Parenting,4.300000


In [33]:
df3[~df3['Type'].isin(["Paid", "Free"])] 

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",NaN,NaN


In [34]:
print("Zero price apps: {}\nFree type apps: {}\nNon-zero price apps: {}\nPaid type apps: {}"\
      .format(sum(df3["Price"] == "0"), 
              sum(df3["Type"] == "Free"), 
              sum(df3["Price"] != "0"),
              sum(df3["Type"] == "Paid")))
df3[(df3["Price"] == "0") & (df3["Type"] != "Free")]

Zero price apps: 8906
Free type apps: 8905
Non-zero price apps: 754
Paid type apps: 754


,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
App,,,,,,,,,,,,
Command & Conquer: Rivals,FAMILY,NaN,0,NaN,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",NaN,NaN


In [35]:
df6 = df3.assign(Price_amt = lambda d: d.Price.str.extract('(\d+(\.\d+)?)')[0].astype(float))
df6

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Price_amt
App,,,,,,,,,,,,,
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3,0.0
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3,0.0
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,26214400.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2,0.0
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2936012.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4,0.0
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5872025.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3,0.0
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19922944.0,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3,0.0
Infinite Painter,ART_AND_DESIGN,4.1,36815,30408704.0,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2,0.0
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,34603008.0,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0,0.0
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3250585.6,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3,0.0


In [36]:
incomes = df6.Price_amt * df6.Installs
incomes.sort_values(ascending = False)

App
Minecraft                                             69900000.0
I am rich                                             39999000.0
I Am Rich Premium                                     19999500.0
Hitman Sniper                                          9900000.0
Grand Theft Auto: San Andreas                          6990000.0
Facetune - For Free                                    5990000.0
Sleep as Android Unlock                                5990000.0
DraStic DS Emulator                                    4990000.0
I'm Rich - Trump Edition                               4000000.0
I am Rich Plus                                         3999900.0
💎 I'm rich                                             3999900.0
I Am Rich                                              3899900.0
I am rich VIP                                          2999900.0
Weather Live                                           2995000.0
Card Wars - Adventure Time                             2990000.0
Threema              

In [37]:
gps_ur.head()

,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
App,,,,
10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
10 Best Foods for You,NaN,NaN,NaN,NaN
10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [38]:
len(gps_ur.index.unique())

1074

In [39]:
gps_ur.reset_index().groupby("App").agg({'Sentiment_Polarity': ['min', 'max']}).sort_values(("Sentiment_Polarity", "min"), ascending = False)

Sentiment_Polarity  \
                                                                  min   
App                                                                     
HomeWork                                                     1.000000   
Google Slides                                                0.933333   
Daily Workouts - Exercise Fitness Routine Trainer            0.800000   
Bed Time Fan - White Noise Sleep Sounds                      0.781250   
GPS Map Free                                                 0.700000   
Google Primer                                                0.700000   
Best Ovulation Tracker Fertility Calendar App Glow           0.595313   
All-in-One Mahjong 3 FREE                                    0.500000   
GPS Speedometer and Odometer                                 0.450000   
CallApp: Caller ID, Blocker & Phone Call Recorder            0.390741   
Brightest Flashlight Free ®                                  0.390000   
Drawing for Kids Learning Games for Toddlers age 3           0.386667   
Caller ID +                                                  0.350000   
FlipaClip - Cartoon animation                                0.337500   
Best Fiends - Free Puzzle Game                               0.325000   
Amazon Prime Video                                           0.322222   
Google+                                                      0.319444   
Calendar+ Schedule Planner App                               0.319071   
Google Trips - Travel Planner                                0.300000   
Apartment Decorating Ideas                                   0.245000   
Google Translate                                             0.188333   
Calculator - unit converter                                  0.183333   
CM Flashlight (Compass, SOS)                                 0.150000   
3D Live Neon Weed Launcher                                   0.136364   
Calculator Plus Free                                         0.133333   
Caf - Mon Compte                                             0.100000   
Couple - Relationship App                                    0.100000   
Draw a Stickman: EPIC 2                                      0.100000   
Daniel Tiger for Parents                                     0.072222   
7 Day Food Journal Challenge                                 0.070000   
...                                                               ...   
Girly Wallpapers Backgrounds                                      NaN   
Glam - Premium Dating App                                         NaN   
GlassesOff                                                        NaN   
Glide - Video Chat Messenger                                      NaN   
Glow Hockey                                                       NaN   
Glowing Flowers Live Wallpaper                                    NaN   
Google Allo                                                       NaN   
Google Analytics                                                  NaN   
Google Assistant                                                  NaN   
Google Assistant Go                                               NaN   
Google Calendar                                                   NaN   
Google Chrome: Fast & Secure                                      NaN   
Google Classroom                                                  NaN   
Google Docs                                                       NaN   
Google Drive                                                      NaN   
Google Play Books                                                 NaN   
Google Play Games                                                 NaN   
Google Play Movies & TV                                           NaN   
Google Sheets                                                     NaN   
HBO NOW: Stream TV & Movies                                       NaN   
HOTEL DEALS                                                       NaN   
HSL - Tickets, route planner and inf